In [2]:
#importy
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import re
from scipy import stats
import numpy as np

# Tworzenie tabeli z zebranymi ogłoszeniami.

In [27]:
def poczatek():
    url1 = input('Wprowadź link do strony z wynikami Twojego wyszukiwania. Przejdę przez wszystkie dostępne strony. ')
    strona = requests.get(url1)
    soup = bs(strona.text,'lxml')
    
    return soup, url1

#soup, url = poczatek()

In [99]:
soup, url1 = poczatek()

Wprowadź link do strony z wynikami Twojego wyszukiwania. Przejdę przez wszystkie dostępne strony. https://www.otodom.pl/wynajem/mieszkanie/?search%5Bfilter_float_m%3Ato%5D=50&search%5Bfilter_enum_rooms_num%5D%5B0%5D=2&search%5Bfilter_enum_rooms_num%5D%5B1%5D=3&search%5Bfilter_enum_rooms_num%5D%5B2%5D=1&locations%5B0%5D%5Bregion_id%5D=12&locations%5B0%5D%5Bsubregion_id%5D=294&locations%5B0%5D%5Bcity_id%5D=166&locations%5B1%5D%5Bregion_id%5D=12&locations%5B1%5D%5Bsubregion_id%5D=445&locations%5B1%5D%5Bcity_id%5D=178&locations%5B2%5D%5Bregion_id%5D=12&locations%5B2%5D%5Bsubregion_id%5D=306&locations%5B2%5D%5Bcity_id%5D=172


In [104]:
def zakres():    
    p2 = soup.find('form', {"id":'pagerForm'})
    p2 = p2.find_all('a',"")

    strony = []

    for a in p2:
        strona = a.text
        if strona != "":
            strona = int(strona)
            strony.append(strona)
        else:
            pass
    strona_max = max(strony)
    strona_min = min(strony)

    return(strona_min, strona_max)

#strona_min, strona_max = zakres()

In [105]:
strona_min,strona_max = zakres()

In [106]:
def limit_wyszukiwan(strona_max):

    limit_wyszukiwania = 0
    
    if strona_max > 50:
        limit_wyszukiwania =+ 50
    else:
        limit_wyszukiwania =+ strona_max
    
    return limit_wyszukiwania

In [107]:
limit_wyszukiwania = limit_wyszukiwan(strona_max)
limit_wyszukiwania

36

In [108]:
def parametry():
    
    #tworze główny data frame
    df = pd.DataFrame(columns=['tytul','typ','miejsce','pokoje','pietro','pietra_budynku','rok_budowy',
                               'powierzchnia','cena',
                               'czynsz_osobny','kaucja','wynajmujacy','link'])
    
    #listy na wartości z ogłoszeń + tworzenie kolejnych iteracji linku
    for i in range(strona_min,limit_wyszukiwania+1):
        
        l_tytul = []
        l_typ = []
        l_miejsce = []
        l_pokoje = []
        l_powierzchnia = []
        l_cena = []
        l_wynajmujacy = []
        
        
        link_oferta = []
        
        l_czynsz_dod = []
        l_kaucja =[]
        l_pietro = []
        l_pietra = []
        l_rok_bud = []
        
        url = url1+ f'&page={i}'
        strona = requests.get(url)
        soup = bs(strona.text,'lxml')

        oferty = soup.find_all('article')
    
        #pętla ściągania ofert z pojedynczej strony wyszukiwania
        for oferta in oferty:

            tytul = oferta.find('span', {"class":"offer-item-title"}).text
            l_tytul.append(tytul)

            typ = oferta.find('span', {"class": "hidden-xs"}).text.strip().rstrip(":")
            l_typ.append(typ)

            miejsce = oferta.find('p', {"class": "text-nowrap"}).text
            miejsce = miejsce[miejsce.find(":",0,-1)+2:]
            l_miejsce.append(miejsce)

            pokoje = int(oferta.find('li', {'class': 'offer-item-rooms hidden-xs'}).text[0])
            l_pokoje.append(pokoje)

            powierzchnia = int(re.findall(r'\d+',oferta.find('li', {"class": "hidden-xs offer-item-area"}).text)[0])
            l_powierzchnia.append(powierzchnia)

            cena = int(re.findall(r'\d+',re.sub(r'\s+','',oferta.find('li', {'class': "offer-item-price"}).text.strip()))[0])
            l_cena.append(cena)

            wynajmujacy = oferta.find('ul', {'class': "params-small clearfix hidden-xs"}).text.strip()
            l_wynajmujacy.append(wynajmujacy)
            
            #url do oferty
            url2 = oferta.find_all('a')[0].get('href')
            link_oferta.append(url2)
            
            #pobieranie danych dodatkowych z strony konkretnego ogłoszenia.
            strona2 = requests.get(url2)
            soup2 = bs(strona2.text,'lxml')

            try:
                pietro = re.findall(r'\d+',soup2.find_all('div', {'aria-label': 'Piętro'})[0].text)
                l_pietro.append(pietro[0])
            except:
                l_pietro.append(0)

            try:
                czynsz_dod = re.findall(r'\d+',soup2.find_all('div', {'aria-label': 'Czynsz - dodatkowo'})[0].text)
                l_czynsz_dod.append(czynsz_dod[0])
            except:
                l_czynsz_dod.append(0)

            try:
                kaucja = re.findall(r'\d+',soup2.find_all('div', {'aria-label': 'Kaucja'})[0].text.replace(" ",""))
                l_kaucja.append(kaucja[0])    
            except:
                l_kaucja.append(0)

            try:
                rok_budowy = re.findall(r'\d+',soup2.find_all('div', {'aria-label': 'Rok budowy'})[0].text)
                l_rok_bud.append(rok_budowy[0])
            except:
                l_rok_bud.append(0)

            try:
                liczba_pieter = re.findall(r'\d+',soup2.find_all('div', {'aria-label': 'Liczba pięter'})[0].text)
                l_pietra.append(liczba_pieter[0])
            except:
                l_pietra.append("NaN")

            
        #tworze data frame z jednej strony wyszukiwania
        df1 = pd.DataFrame({'tytul':l_tytul, 
                            "typ":l_typ,
                            "miejsce": l_miejsce,
                            "pokoje": l_pokoje,
                            "pietro": l_pietro,
                            "pietra_budynku": l_pietra,
                            "rok_budowy": l_rok_bud,
                            "powierzchnia": l_powierzchnia,
                            "cena": l_cena,
                            "czynsz_osobny": l_czynsz_dod,
                            "kaucja": l_kaucja,
                            "wynajmujacy": l_wynajmujacy,
                            "link": link_oferta}) 

        
        #dodaję df z jednej strony wyszukiwania do df głównego
        ramki = [df,df1]
        df = pd.concat(ramki)
    
    
    #dopasowuje automatycznie typy danych
    df = df.convert_dtypes()
    
    #wyciągam miasto z tekstu
    df['miasto'] = df['miejsce'].str.split(',').str[0]
    
    #dodaje typy danych kategoryczne + float
    df['miasto'] = df['miasto'].astype('category')
    df['typ'] = df['typ'].astype('category')
    df['miejsce'] = df['miejsce'].astype('category')
    df['wynajmujacy'] = df['wynajmujacy'].astype('category')
    df['cena'] = df['cena'].astype('int')
    df['czynsz_osobny']= df['czynsz_osobny'].astype('int')
    df['kaucja'] = df['kaucja'].astype('int')
    
    #dodaję sumy dla opłat
    df['suma_oplat'] = df[['cena','czynsz_osobny']].sum(axis = 1)
    df['pierwsza_oplata'] = df[['cena','czynsz_osobny','kaucja']].sum(axis = 1)
    
    #zmiana kolejności kolumn
    df = df[['tytul','typ','miasto','miejsce','pokoje','pietro','pietra_budynku','rok_budowy',
             'powierzchnia','cena','czynsz_osobny','kaucja','suma_oplat','pierwsza_oplata',
             'wynajmujacy','link']]
    
    #usuwanie duplikatów wierszy
    df = df.drop_duplicates()
    df = df.reset_index(drop = True)
    
    #usuwanie wartości bardzo odstających z ceny
    df['z_score'] = np.abs(stats.zscore(df['cena'])<3)
    df = df[df['z_score']==True]
    
    return df

In [109]:
tabela = parametry()

In [110]:
tabela

,tytul,typ,miasto,miejsce,pokoje,pietro,pietra_budynku,rok_budowy,powierzchnia,cena,czynsz_osobny,kaucja,suma_oplat,pierwsza_oplata,wynajmujacy,link,z_score
0,Nowy Brynów mieszkanie wynajem 43m do wykończenia,Mieszkanie na wynajem,Katowice,"Katowice, Brynów-Zgrzebnioka, Brynów",2,1,7,2020,43,1000,430,3000,1430,4430,Oferta prywatna,https://www.otodom.pl/pl/oferta/nowy-brynow-mi...,True
1,Katowice Koszutka - wysoki standard,Mieszkanie na wynajem,Katowice,"Katowice, Koszutka",2,1,NaN,0,35,1300,361,1800,1661,3461,Obsługa zdalna\n\n ...,https://www.otodom.pl/pl/oferta/katowice-koszu...,True
2,"Nowy Brynów, urządzone nowe mieszkanie",Mieszkanie na wynajem,Katowice,"Katowice, Brynów-Zgrzebnioka, Brynów",2,3,7,2020,43,1850,0,3800,1850,5650,Oferta prywatna,https://www.otodom.pl/pl/oferta/nowy-brynow-ur...,True
3,Nowoczesny apartament MAXX w centrum z parkingiem,Mieszkanie na wynajem,Bielsko-Biała,"Bielsko-Biała, Centrum",1,3,4,2019,30,1650,250,3500,1900,5400,Oferta prywatna,https://www.otodom.pl/pl/oferta/nowoczesny-apa...,True
4,"ul. Jagielońska, 2 pok. po remoncie, Bez prowizji",Mieszkanie na wynajem,Katowice,"Katowice, Śródmieście",2,1,NaN,0,35,1100,0,0,1100,1100,Obsługa zdalna\n\n ...,https://www.otodom.pl/pl/oferta/ul-jagielonska...,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
957,"Mieszkanie, 23,50 m², Tychy",Mieszkanie na wynajem,Tychy,"Tychy, C",1,1,3,0,23,1200,0,0,1200,1200,"""MARKSON - NIERUCHOMOŚCI"" mgr Rafał Biegun",https://www.otodom.pl/pl/oferta/mieszkanie-23-...,True
958,Katowice Centrum ok. Opolska 2 pokoje 40 m2,Mieszkanie na wynajem,Katowice,"Katowice, Śródmieście",2,0,3,0,40,1200,0,2400,1200,3600,TENERE Nieruchomości,https://www.otodom.pl/pl/oferta/katowice-centr...,True
959,Centrum ok. Opolska tanio kawalerka po remoncie,Mieszkanie na wynajem,Katowice,"Katowice, Centrum",1,0,3,0,23,900,0,1500,900,2400,TENERE Nieruchomości,https://www.otodom.pl/pl/oferta/centrum-ok-opo...,True
960,"Mieszkanie, 41 m², Katowice",Mieszkanie na wynajem,Katowice,"Katowice, Śródmieście",2,0,4,1910,41,1200,0,2580,1200,3780,TENERE Nieruchomości,https://www.otodom.pl/pl/oferta/mieszkanie-41-...,True


In [69]:
tabela.dtypes

tytul               string
typ               category
miasto            category
miejsce           category
pokoje               Int64
pietro              object
pietra_budynku      string
rok_budowy          object
powierzchnia         Int64
cena                 int32
czynsz_osobny        int32
kaucja               int32
wynajmujacy       category
link                string
z_score               bool
dtype: object

# Analiza i tabele

In [111]:
#importy bibliotek
import plotly.express as px


In [114]:
#podsumowanie zebranych ofert

def podsumowanie_ofert():
    
    #podsu = tabela.groupby(['miasto','pokoje']).count().reset_index()
    
    podsu = tabela.groupby(['miasto','pokoje']).agg({'cena':['mean', 'max', 'min'],
                         'suma_oplat':['mean','max','min'],
                         'powierzchnia':'mean', 
                         'tytul':'count'})
    
    print("Podsumowanie ofert z Twojego wyszukiwania OTODOM: ")
    
    return podsu

In [115]:
podsumowanie = podsumowanie_ofert()
podsumowanie

Podsumowanie ofert z Twojego wyszukiwania OTODOM: 


cena               suma_oplat              \
                             mean   max   min         mean   max   min   
miasto        pokoje                                                     
Bielsko-Biała 1       1257.238095  1800   700  1410.404762  1900   700   
              2       1606.616667  2490  1000  1836.733333  2790  1100   
              3       1584.285714  2600  1000  1748.571429  2600  1300   
Katowice      1       1141.625000  2300   271  1205.656250  2450   271   
              2       1662.524123  3000   650  1805.813596  3170   800   
              3       1873.714286  2500  1200  1945.000000  2600  1400   
Tychy         1       1313.181818  1590   800  1324.090909  1590   800   
              2       1683.255814  3000  1200  1824.023256  3000  1450   
              3        500.000000   500   500   500.000000   500   500   

                     powierzchnia tytul  
                             mean count  
miasto        pokoje                     
Bielsko-Biała 1         30.142857    42  
              2         42.166667    60  
              3         46.285714     7  
Katowice      1         26.628125   320  
              2         41.396930   456  
              3         48.095238    21  
Tychy         1         32.272727    11  
              2         40.697674    43  
              3         42.000000     1

## Wykresy

In [116]:
oferty_miasto = tabela.groupby('miasto').count()
oferty_miasto['liczba_ofert'] = oferty_miasto['tytul']
oferty_miasto = oferty_miasto.reset_index()[['miasto','tytul']]
oferty_miasto.columns = ['miasto','liczba_ofert']
oferty_miasto

of_miasto_wykres = px.pie(oferty_miasto,
                         values = 'liczba_ofert',
                         names = 'miasto',
                         title = "Liczba ofert w danym mieście."
                         )

of_miasto_wykres

In [117]:
powierzchnia_miasto = px.bar(tabela,
                      y = 'powierzchnia',
                      x = 'miasto',
                      title = 'Dostępna powierzchnia z Twojego wyszukiwania wg. miasta. [m2]',
                      )
powierzchnia_miasto.show()

In [118]:
pokoje_box = px.box(tabela, 
                    y = 'cena',
                    points = 'all',
                    hover_data = ['powierzchnia','pokoje','miasto'],
                    title = 'Rozrzut wszystkich ofert wg. ceny.- przybliż wykres w interesujące miejsce',
                    
                   )
pokoje_box

In [119]:
pokoje_suma_oplat = px.box(tabela, 
                    y = 'suma_oplat',
                    points = 'all',
                    hover_data = ['powierzchnia','pokoje','miasto'],
                    title = 'Rozrzut wszystkich ofert wg. sumy opłat msc.',
                    
                   )
pokoje_suma_oplat

In [120]:
powierzchnia_cena_scatter = px.scatter(tabela,
                           y = 'cena',
                           x = 'powierzchnia',
                           title = 'Rozrzut cenowy ofert.', 
                           color = 'miasto')

powierzchnia_cena_scatter

In [121]:
cena_pokoje_box = px.box(tabela,
                                y = 'cena',
                                x = 'pokoje',
                                color = 'miasto',
                                title = 'Rozrzut cenowy ofert (wg l.pokojów)')

cena_pokoje_box

In [122]:
suma_oplat_box = px.box(tabela,
                                y = 'suma_oplat',
                                x = 'pokoje',
                                color = 'miasto',
                                title = 'Rozrzut cenowy pełnej opłaty per msc (wg l.pokojów)')

suma_oplat_box

In [124]:
powierzchnia_suma_oplat_scatter = px.scatter(tabela,
                           y = 'suma_oplat',
                           x = 'powierzchnia',
                           title = 'Rozrzut sum oplat za mieszkanie.', 
                           color = 'miasto')

powierzchnia_suma_oplat_scatter

In [126]:
kaucja_box = px.box(tabela,
                                y = 'kaucja',
                                x = 'pokoje',
                                color = 'miasto',
                                title = 'Rozrzut cenowy kaucji wstepnej')

kaucja_box

In [127]:
pierwsza_oplata_box = px.box(tabela,
                                y = 'pierwsza_oplata',
                                x = 'pokoje',
                                color = 'miasto',
                                title = 'Rozrzut cenowy kaucji wstepnej')

pierwsza_oplata_box